In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/scratch/ewalt/pdm/rs-uncertainty")
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from src.metrics import StratifiedRCU
from src.viz import *
from pathlib import Path
import rasterio
import fiona
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import trange
import random
import yaml
import fiona
import rasterio.warp
import rasterio.features
sns.set()
sns.set_style("whitegrid")
random.seed(123)

RESDIR = "results/cloud_exp/2023-06-20_16-14-11" #"results/cloud_exp/2023-05-26_15-55-46"
S2DIR = "gee_data/reprojected/"
S2REPRDIR = "gee_data/reprojected_dirs"
GTDIR = "assets/data/preprocessed"
SANITYRESDIR = "results/cloud_exp/2023-05-31_11-23-56_sanity_check" # results
SANITYS2DIR = "assets/data/sentinel_data/s2_reprojected" # s2 reprojected
SANITYS2REPRDIR = "gee_data/sanity_check/" # restructured s2 reprojected
SPLITMASKDIR = "assets/data/split_masks/" # split masks
SHAPEFILES = ['assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 'assets/data/ALS_projects_Dz_all_norway.shp']
STATSFILE = "data/2023-04-05_18-58-33_baseline/stats.yaml"
EMPIRICAL_CP_THRESHOLD = 7.9

with open(STATSFILE, "r") as f:
    stats = yaml.safe_load(f)
TRAINMEANS = stats["labels_stats"]["mean"]
TRAINSTDS = stats["labels_stats"]["std"]
for i in [2,4]:
    TRAINMEANS[i] /= 100
    TRAINSTDS[i] /= 100
    
VARIABLES = ['P95', 'MeanH', 'Dens', 'Gini', 'Cover']

# Experiment result directories
result_dirs = [p.path for p in os.scandir(RESDIR) if os.path.exists(os.path.join(p.path,"rcu.json"))]
outliers = [os.path.join(RESDIR, f"1023_{d}") for d in [
    "20180503T104019", # index: 3, avgcp: 42, all white
    "20180620T105031", # index: 6, avgcp: 0., all white
]]
result_dirs = [r for r in result_dirs if not any(r.__contains__(o) for o in outliers)]
len(result_dirs)

## Random 4 images

In [ ]:
fig, axs = plt.subplots(ncols=4,nrows=1,figsize=(12,6))
axs = axs.flatten()
def showOnAxis(d, ax):
    p = getPaths(d, s2repr_dirs=S2REPRDIR, returns=["img"])
    rgb = loadRaster(p, transpose_order=(1,2,0), clip_range=(100,2000), bands=[4,3,2])
    ax.imshow(rgb)
    ax.set_axis_off()
for d in os.scandir(RESDIR):
    dd = d.path
    if "20180630T" in dd: showOnAxis(dd, axs[0])
    elif "20180627" in dd: showOnAxis(dd, axs[1])
    elif "20180707" in dd: showOnAxis(dd, axs[2])
    elif "20180824" in dd: showOnAxis(dd, axs[3])
plt.tight_layout()
savefigure(fig, "images/albecker/random_4_cond")
plt.show()

## Empirical Cp threshold

In [ ]:
def computeEmpiricalCpDistribution(config):
    all_cps = np.arange(0, 101)
    cps_histogram = np.zeros((100,))
    selected_cps_histogram = np.zeros((100,))
    cnt = 0
    for i, gt_file_path in enumerate(Path(config["gt_dir"]).glob("*.tif")):
        project_id = gt_file_path.stem
        if (project_id in config["projects"] and 
            os.path.exists(os.path.join(config["s2_reprojected_dir"], project_id))):
            cnt += 1
            print(f"Processing {project_id} [{cnt}/{len(config['projects'])}]")
            # get gt_file and valid_mask
            gt_file = rasterio.open(gt_file_path)
            valid_mask = gt_file.read_masks(1)
            # read split mask
            with rasterio.open(os.path.join(config["split_mask_dir"], f"{project_id}.tif")) as f:
                split_mask = f.read(1).astype("float16")
            assert valid_mask.shape==split_mask.shape
            # rasterized polygon and gt date
            project_shape_collections = [fiona.open(p) for p in config['project_shapefiles']]
            # create the shape ("polygon") associated to the project 
            polygon, crs, gt_date = None, None, None
            for collection in project_shape_collections:
                try:
                    polygon = [s['geometry'] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
                    crs = collection.crs
                    gt_date = [s["properties"]["PUB_DATO"] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
                    gt_date = parse_gt_date(gt_date)
                    break
                except IndexError: pass 
            if polygon is None: print("No polygon found")
            polygon = rasterio.warp.transform_geom(src_crs=crs, dst_crs=gt_file.crs, geom=polygon)
            rasterized_polygon = rasterio.features.rasterize(
                [(polygon, 1)],
                out_shape=gt_file.shape,
                transform=gt_file.transform,
                fill=0,
                dtype='uint8'
            )
            # read s2 images
            parse_date = lambda x: datetime.strptime(x, '%Y%m%d')
            s2_images = []
            for img_path in Path(os.path.join(config["s2_reprojected_dir"], project_id)).glob("*.tif"):
                with rasterio.open(img_path) as fh:
                    s2_images.append((fh.read(fh.indexes), parse_date(img_path.stem.split('_')[3].split('T')[0])))
            # Get valid centers
            shape = gt_file.shape
            patch_half = config["patch_size"]//2
            for i in trange(patch_half, gt_file.shape[0] - patch_half):
                for j in range(patch_half, gt_file.shape[1] - patch_half):
                    i_slice = slice(i - patch_half, i + patch_half + 1)
                    j_slice = slice(j - patch_half, j + patch_half + 1)
                    is_train_split = (split_mask[i_slice, j_slice] == 0).all()
                    is_in_polygon = (rasterized_polygon[i_slice, j_slice] == 1).all()
                    # Ignore pixels that span out of train split and don't lie completly in their polygon
                    if is_train_split and is_in_polygon and valid_mask[i,j]:
                        for s2_image, s2_date in s2_images:
                            cps = s2_image[-1, i_slice, j_slice].reshape((-1))
                            cp_histcount, _ = np.histogram(cps, all_cps)
                            cps_histogram += cp_histcount
                            if (s2_image[-1, i_slice, j_slice] > config['cloud_prob_threshold']).sum() \
                                / config['patch_size']**2 > config['cloudy_pixels_threshold']:
                                continue
                            else:
                                selected_cps_histogram += cp_histcount
            # close gt file
            gt_file.close()
    cp_df = pd.DataFrame({
        "cp": np.arange(0, 100),
        "cp_count": cps_histogram,
        "selected": selected_cps_histogram,
        "rejected": cps_histogram-selected_cps_histogram,
    })
    cp_df = cp_df.assign(
        selection_probability=cp_df.selected/cp_df.cp_count,
        rejection_probability=cp_df.rejected/cp_df.cp_count
    )
    return cp_df

In [ ]:
with open("config/create_dataset/baseline.yaml", "r") as f: 
    baseline_config = yaml.safe_load(f)
if os.path.exists(os.path.join("computed_resources", "train_set_pixels_cps.json")):
    cp_df = pd.read_json(os.path.join("computed_resources", "train_set_pixels_cps.json"))
else:
    cp_df = computeEmpiricalCpDistribution(baseline_config)
    cp_df.to_json(os.path.join("computed_resources", "train_set_pixels_cps.json"))

In [ ]:
showEmpiricalSelectionRejectionProbs(
    cp_df,
    EMPIRICAL_CP_THRESHOLD,
    interpolate=True,
    figsize=(12,5),
    save_name="images/albecker/empirical_selection_rejection_proba"
)

## Validity masks

In [ ]:
project_id = "1023"
d = os.path.join(RESDIR, '1023_20180526T105029')
gt_path, img_path = getPaths(d, s2repr_dirs=S2REPRDIR, gt_dir=GTDIR, returns=["gt", "img"])
with rasterio.open(gt_path) as f:
    gts = []
    gt_mask = f.read_masks(1)//255
    for i in range(1,f.count+1):
        gt = f.read(i)
        gt[gt_mask==0] = np.nan
        gt[gt>0] = 100
        gts.append(gt)
    gt = np.nanmean(np.stack(gts, axis=0), axis=0)
    project_shape_collections = [fiona.open(p) for p in baseline_config['project_shapefiles']]
    # create the shape ("polygon") associated to the project 
    polygon, crs, gt_date = None, None, None
    for collection in project_shape_collections:
        try:
            polygon = [s['geometry'] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
            crs = collection.crs
            gt_date = [s["properties"]["PUB_DATO"] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
            gt_date = parse_gt_date(gt_date)
            break
        except IndexError: pass 
    if polygon is None: print("No polygon found")
    polygon = rasterio.warp.transform_geom(src_crs=crs, dst_crs=f.crs, geom=polygon)
    rasterized_polygon = rasterio.features.rasterize(
        [(polygon, 1)],
        out_shape=f.shape,
        transform=f.transform,
        fill=0,
        dtype='float32'
    )
# selected=1, rejected=-1
rasterized_polygon[rasterized_polygon==0] = -1
rasterized_polygon[rasterized_polygon==1] = 1
gt[~np.isnan(gt)] = 1
gt[np.isnan(gt)] = -1
rgb = loadRaster(img_path, bands=[4,3,2], clip_range=(100, 2000), transpose_order=(1,2,0))
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 3.5))
axs[0].imshow(rgb)
axs[1].imshow(rgb)
axs[2].imshow(rgb)
sns.heatmap(rasterized_polygon, ax=axs[1], cbar=False, cmap="magma", alpha=0.6)
sns.heatmap(gt, ax=axs[2], cbar=False, cmap="bwr_r", alpha=0.6)
for ax in axs: ax.set_axis_off()
plt.tight_layout()
plt.savefig("images/albecker/1023_validity_mask_and_polygon.png", dpi=300)
# plt.savefig("images/albecker/1023_validity_mask_and_polygon.pdf", dpi=72)
plt.show()

In [ ]:
# valid mask for other project for report
project_id = "805"
gt_path = "assets/data/preprocessed/805.tif"
img_path = "assets/data/sentinel_data/s2_reprojected/805/805_S2B_MSIL2A_20170720T105029_N9999_R051_T32VNN_20201207T094930.tif"
with rasterio.open(gt_path) as f:
    gts = []
    gt_mask = f.read_masks(1)//255
    for i in range(1,f.count+1):
        gt = f.read(i)
        gt[gt_mask==0] = np.nan
        gt[gt>0] = 100
        gts.append(gt)
    gt = np.nanmean(np.stack(gts, axis=0), axis=0)
    project_shape_collections = [fiona.open(p) for p in baseline_config['project_shapefiles']]
    # create the shape ("polygon") associated to the project 
    polygon, crs, gt_date = None, None, None
    for collection in project_shape_collections:
        try:
            polygon = [s['geometry'] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
            crs = collection.crs
            gt_date = [s["properties"]["PUB_DATO"] for s in collection if s['properties']['kv_id'] == int(project_id)][0]
            gt_date = parse_gt_date(gt_date)
            break
        except IndexError: pass 
    if polygon is None: print("No polygon found")
    polygon = rasterio.warp.transform_geom(src_crs=crs, dst_crs=f.crs, geom=polygon)
    rasterized_polygon = rasterio.features.rasterize(
        [(polygon, 1)],
        out_shape=f.shape,
        transform=f.transform,
        fill=0,
        dtype='float32'
    )
# selected=1, rejected=-1
rasterized_polygon[rasterized_polygon==0] = -1
rasterized_polygon[rasterized_polygon==1] = 1
gt[~np.isnan(gt)] = 1
gt[np.isnan(gt)] = -1
rgb = loadRaster(img_path, bands=[4,3,2], clip_range=(100, 2000), transpose_order=(1,2,0))
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 8))
axs[0].imshow(rgb)
axs[1].imshow(rgb)
axs[2].imshow(rgb)
sns.heatmap(rasterized_polygon, ax=axs[1], cbar=False, cmap="magma", alpha=0.6)
sns.heatmap(gt, ax=axs[2], cbar=False, cmap="bwr_r", alpha=0.6)
for ax in axs: ax.set_axis_off()
plt.tight_layout()
plt.savefig("images/albecker/805_validity_mask_and_polygon.png", dpi=300)
# plt.savefig("images/albecker/1023_validity_mask_and_polygon.pdf", dpi=72)
plt.show()